In [7]:
# En este archivo definimos las funciones que empleará nuestro sistema de web_scrapping

# Declaramos las librerías que usaremos
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException


# Nuestro dataset estará formado por una lista (documento) de listas (filas), que posteriormente
# transformaremos a formato CSV. Rellenamos la primera fila con los nombres de las cabeceras
# (los campos "*_upper" y "*_lower" se refieren a los márgenes de error superior e inferior, respectivamente)
dataset = [['name', 'version', 'release', 'gps', 'mass_1', 'mass_1_upper', 'mass_1_lower', 'mass_2', 'mass_2_upper',
            'mass_2_lower', 'network_snr', 'network_snr_upper', 'network_snr_lower', 'distance', 'distance_upper',
            'distance_lower', 'chi_eff', 'chi_eff_upper', 'chi_eff_lower', 'total_mass', 'total_mass_upper', 
            'total_mass_lower', 'chirp_mass', 'chirp_mass_upper', 'chirp_mass_lower', 'detector_frame_chirp_mass', 
            'detector_frame_chirp_mass_upper', 'detector_frame_chirp_mass_lower', 'redshift', 'redshift_upper', 
            'redshift_lower', 'false_alarm_rate', 'p_astro', 'final_mass', 'final_mass_upper', 'final_mass_lower']]


def execute_form(fecha_inicio, fecha_fin):
    """Función que rellena y ejecuta el formulario"""
    # Declaramos nuestro navegador
    driver = webdriver.Chrome()

    # Abrimos nuestra web
    driver.get('https://www.gw-openscience.org')

    # Seleccionamos en el menú eventos y catálogos
    elem = driver.find_element(By.XPATH, "//a[contains(text(), 'Events and Catalogs')]")   

    # Hacemos Click
    driver.execute_script("arguments[0].click();", elem)
    # Esperamos a que cargue
    driver.implicitly_wait(10)

    # Pulsamos el botón query de la ventana que acaba de cargar
    elem2 = driver.find_element(By.XPATH, "//button[text()='Query']").click() 
    # Esperamos a que se cargue el formulario
    driver.implicitly_wait(10)

    # Vamos a rellenar el formulario
    # Comenzamos por los selectores
    select = Select(driver.find_element(By.ID,'mul-release-sel'))

    # Vamos a seleccionar elementos por su nombre, son las diferentes releases
    select.select_by_visible_text('GWTC-1-marginal')
    select.select_by_visible_text('GWTC-1-confident')
    select.select_by_visible_text('O3_IMBH_marginal')
    select.select_by_visible_text('GWTC-2')
    select.select_by_visible_text('GWTC-2.1-marginal')
    select.select_by_visible_text('GWTC-2.1-confident')
    select.select_by_visible_text('GWTC-3-marginal')
    select.select_by_visible_text('GWTC-3-confident')
    select.select_by_visible_text('Initial_LIGO_Virgo')
    
    # Ahora metemos las fechas "desde" y "hasta"
    imputFecha1 = driver.find_element(By.XPATH,"//div[@id='min-datetime-tbox']/input")
    imputFecha1.send_keys(fecha_inicio+'T00:00:00')  
    imputFecha2 = driver.find_element(By.XPATH,"//div[@id='max-datetime-tbox']/input")
    imputFecha2.send_keys(fecha_fin+'T00:00:00')  
    
    # Ahora marcamos el checkbox para que nos den la última versión de las detecciones
    # De este modo no tendremos repetidos
    checkbox = driver.find_element(By.XPATH,"//label[@id='version-tbox']/input")
    # Hacemos Click para marcar el checkbox
    driver.execute_script("arguments[0].click();", checkbox)
    
    # Ya tenemos todo, pulsamos el botón submit para generar la tabla con los datos
    # Localizamos el botón
    boton = driver.find_element(By.ID, 'submit-query-btn')
    # Lo pulsamos
    driver.execute_script("arguments[0].click();", boton)
    # Esperamos a que se cargue la tabla
    driver.implicitly_wait(10)
    
    # Cuidado que no selecciona todas las columnas, debemos seleccionar lo que deseamos ver
    # Pulsamos el botón para desplegar el selector de columnas
    desplegable = driver.find_element(By.CLASS_NAME ,'tablesaw-columntoggle-btnwrap').click()
    
    # Ahora ya vemos los selectores de columnas, comprobaremos que estén todos seleccionados,
    # pulsando aquellos que no lo estén.
    # Creamos una función para evitar la duplicación de código en esta parte
    def click_checkbox(checkbox_text):
        # Localizamos el checkbox
        checkbox = driver.find_element(By.XPATH, f"//label[text()='{checkbox_text}']/input")
        # Comprobamos si está checkeado
        if checkbox.get_attribute("checked") != "true":
            # Hacemos Click para marcar el checkbox
            driver.execute_script("arguments[0].click();", checkbox)
    
    # Utilizamos dicha función para marcar todas las casillas (columnas) deseadas
    casillas = ['Version', 'Release', 'GPS', 'Mass 1 (M☉)', 'Mass 2 (M☉)', 'Network SNR',
                'Distance (Mpc)', 'χeff', 'Total Mass (M☉)', 'Chirp Mass (M☉)', 
                'Detector Frame Chirp Mass (M☉)', 'Redshift', 'False Alarm Rate (yr-1)', 
                'Pastro', 'Final Mass (M☉)']
    for casilla in casillas:
        click_checkbox(casilla)
        
    # Pulsamos el botón para ocultar el desplegable y poder trabajar cómodamente
    desplegable = driver.find_element(By.CLASS_NAME ,'tablesaw-columntoggle-btnwrap').click()
    
    # Ya tenemos la tabla en pantalla, ahora la recorreremos para obtener los datos
    # Primeramente, obtenemos cada una de sus filas (excluyendo la primera, las cabeceras)
    detecciones = driver.find_elements(By.CSS_SELECTOR, "tr")[1:]

    # Por cada detección (fila) trataremos cada una de sus columnas. Algunas de estas columnas
    # merecen un trato especial, como es el caso de las columnas numéricas que pueden tener
    # valores de error superior e inferior. Creamos pues una función para tratarlas:

    def get_value_upper_lower(field):
        # Tomamos el valor del campo, desechando el valor numérico "escondido" usado para ordenar,
        # y los demás posibles valores restantes (errores superior e inferior)
        if '--' in field.text:
            return None, None, None
        try:
            value = field.text.split(' ')[1].split('\n')[0]
        except:
            print(field.text)
            return None, None, None
        # Obtenemos el valor del error superior (si existe)
        try:
            upper = field.find_element(By.CSS_SELECTOR, "sup").text
        except NoSuchElementException:
            upper = None
        # Obtenemos el valor del error inferior (si existe)
        try:
            lower = field.find_element(By.CSS_SELECTOR, "sub").text
        except NoSuchElementException:
            lower = None
        # Retornamos los 3 valores
        return value, upper, lower

    # Hay otro tipo de campos que pueden contener valores con espacios (ej.: "≥ 0.99"). También
    # cuidaremos de tratar estos mediante esta función:

    def get_value_potential_spaces(field):
        # Tomamos el valor del campo, desechando el valor numérico "escondido" usado para ordenar,
        # y tomando todo aquello que siga detrás
        if '--' in field.text:
            return None
        return ' '.join(field.text.split(' ')[1:])

    # Y con esto obtenemos el dataset:

    for d in detecciones:
        # Dividimos la fila en sus columnas
        d = d.find_elements(By.CSS_SELECTOR, "td")
        print([i.text for i in d])
        # Tratamos los valores numéricos con posibles medidas de error
        mass1, mass2, network, distance, hi_eff, total_mass, chirp_mass, detector_frame_chirp_mass, \
        redshift, final_mass = map(get_value_upper_lower, [d[4], d[5], d[6], d[7], d[8], d[9], d[10],
        d[11], d[12], d[15]])
        # Tratamos los valores con posibles espacios en ellos
        false_alarm_rate, p_astro = map(get_value_potential_spaces, [d[13], d[14]])
        # Añadimos la línea correspondiente al dataset, tratando todos sus campos adecuadamente
        # (aplicamos "get_value_upper_lower", "get_value_potential_spaces" y también cuidamos
        # de convertir todos los valores "--" a empty string (""))
        dataset.append(list(map(lambda valor: valor.replace('--', '') if valor else '',
            [d[0].text, d[1].text, d[2].text, d[3].text, 
            mass1[0], mass1[1], mass1[2], 
            mass2[0], mass2[1], mass2[2],
            network[0], network[1], network[2],
            distance[0], distance[1], distance[2],
            hi_eff[0], hi_eff[1], hi_eff[2],
            total_mass[0], total_mass[1], total_mass[2],
            chirp_mass[0], chirp_mass[1], chirp_mass[2],
            detector_frame_chirp_mass[0], detector_frame_chirp_mass[1], detector_frame_chirp_mass[2],
            redshift[0], redshift[1], redshift[2],
            false_alarm_rate, p_astro,
            final_mass[0], final_mass[1], final_mass[2]]
        )))

    return dataset


with open('./test.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # writer.writerows(execute_form('20150101','20221105'))
    writer.writerows(execute_form('20200301','20221105'))


['GW200322_091133', 'v1', 'GWTC-3-confident', '1268903511.3', '34.0 34\n+48\n-18', '14.0 14.0\n+16.8\n-8.7', '6.0 6.0\n+1.7\n-1.2', '3600.0 3600\n+7000\n-2000', '0.24 0.24\n+0.45\n-0.51', '55.0 55\n+37\n-27', '15.5 15.5\n+15.7\n-3.7', '-2.0 --', '0.6 0.60\n+0.84\n-0.30', '140.0 140', '0.61501 0.62', '53.0 53\n+38\n-26']
['GW200316_215756', 'v1', 'GWTC-3-confident', '1268431094.1', '13.1 13.1\n+10.2\n-2.9', '7.8 7.8\n+1.9\n-2.9', '10.3 10.3\n+0.4\n-0.7', '1120.0 1120\n+470\n-440', '0.13 0.13\n+0.27\n-0.10', '21.2 21.2\n+7.2\n-2.0', '8.75 8.75\n+0.62\n-0.55', '-2.0 --', '0.22 0.22\n+0.08\n-0.08', '0.00001 ≤ 1.0e-05', '0.99 ≥ 0.99', '20.2 20.2\n+7.4\n-1.9']
['GW200311_115853', 'v1', 'GWTC-3-confident', '1267963151.3', '34.2 34.2\n+6.4\n-3.8', '27.7 27.7\n+4.1\n-5.9', '17.8 17.8\n+0.2\n-0.2', '1170.0 1170\n+280\n-400', '-0.02 -0.02\n+0.16\n-0.20', '61.9 61.9\n+5.3\n-4.2', '26.6 26.6\n+2.4\n-2.0', '-2.0 --', '0.23 0.23\n+0.05\n-0.07', '0.00001 ≤ 1.0e-05', '0.99 ≥ 0.99', '59.0 59.0\n+4.8\n-3